In [242]:
import pandas as pd
from collections import Counter
import spacy
import pytextrank
import json

# Create comment wise JSON

In [243]:
df = pd.read_json('communitycrit_revised.json')
# df = pd.read_json('env_canada_final.json')

In [244]:
data = []
cid = 0;

for i in df["ideas"]:
    idea_name = i["name"]
    idea_id = i["serial"]
    for t in i["tasks"]:
        for c in t["comments"]:
            data_packet = {
                "comment_id": cid,
                "comment": c["comment"],
                "related_to": (c["question"] if c["task_id"] == 12 else t["name"]),
                "discussion_id": idea_id,
                "discussion_name": idea_name,
                "user_id": c["user_id"],
                "emotion": "",
                "discourse": "",
                "dominant_topic": "",
                "topic_phrase": "",
                "topic_keywords": "",
                "lemmas": ""
            }
            cid += 1
            data.append(data_packet)

In [245]:
emo_df = pd.read_json('cc_preds_emo_bert.json')
# emo_df = pd.read_json('envcanada_preds_emo_bert.json')

emo_df.head(10)

for idx, row in emo_df.iterrows():
    data[idx]["emotion"] = row["emotion"]

In [246]:
disc_df = pd.read_json('cc_preds_bert.json')
# disc_df = pd.read_json('envcanada_preds_bert.json')

disc_df.head(10)

for idx, row in disc_df.iterrows():
    data[idx]["discourse"] = row["discourse"]

In [247]:
topic_df = pd.read_json('cc_sent.json')
# topic_df = pd.read_json('env_sent.json')

for idx, row in topic_df.iterrows():
    data[idx]["dominant_topic"] = row["dominant_topic"]
    data[idx]["topic_keywords"] = row["keywords"].split(", ")

In [248]:
lemma_df = pd.read_json('cc_lemmas.json')
# lemma_df = pd.read_json('env_lemmas.json')

for idx, row in lemma_df.iterrows():
    data[idx]["lemmas"] = row["lemmas"]

In [249]:
def get_keyphrase(prop_rows, t):
    text = ""
    rows = prop_rows.loc[prop_rows['dominant_topic'] == t]
    for c in rows["comment"]:
        text = text + " " + c
        
    nlp = spacy.load("en_core_web_sm")
    tr = pytextrank.TextRank(logger=None)
    nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
    doc = nlp(text)
    
#     print(doc._.phrases, "\n\n")
    
    if (doc._.phrases):
        return doc._.phrases[0].text
    else:
        return rows.iloc[0]["keywords"].split(", ")[0]
    

topic_phrases = []
proposals = Counter(topic_df["idea"])
for p in proposals:
    proposal_wise_topic_phrases = {
        "proposal": p,
    }
    prop_rows = topic_df.loc[topic_df['idea'] == p]
    topics = Counter(prop_rows["dominant_topic"])
    print(topics)
    for t in topics:
        proposal_wise_topic_phrases["topic"] = t
        proposal_wise_topic_phrases["topic_phrase"] = get_keyphrase(prop_rows, t)
        topic_phrases.append(proposal_wise_topic_phrases.copy())

Counter({1: 8, 0: 7, 4: 7, 3: 7, 5: 4, 2: 3})
Counter({1: 12, 0: 11})
Counter({4: 9, 2: 6, 1: 4, 0: 3, 3: 2, 5: 2})
Counter({1: 8, 0: 7, 3: 6, 2: 5})
Counter({0: 8, 2: 6, 4: 5, 1: 4, 3: 3, 5: 2})
Counter({1: 6, 0: 5, 2: 3, 3: 2})
Counter({0: 6, 4: 5, 2: 4, 3: 2, 1: 2, 5: 1})
Counter({6: 6, 2: 6, 4: 4, 1: 3, 0: 2, 3: 2, 5: 2})
Counter({0: 10, 4: 6, 1: 6, 3: 5, 5: 4, 2: 3})
Counter({1: 1})
Counter({0: 11, 1: 10})
Counter({0: 9, 2: 6, 3: 5, 1: 1})
Counter({0: 3, 1: 2, 3: 2})
Counter({0: 6, 3: 5, 1: 5, 2: 3})
Counter({1: 3, 0: 3})
Counter({0: 2, 1: 1})
Counter({1: 2, 0: 2})
Counter({0: 3, 1: 2})
Counter({4: 5, 2: 5, 5: 4, 3: 3, 1: 2, 0: 2, 6: 1, 7: 1})


In [250]:
for idx, row in topic_df.iterrows():
    for tp in topic_phrases:
        if(row["idea"] == tp["proposal"] and row["dominant_topic"] == tp["topic"]):
            data[idx]["topic_phrase"] = tp["topic_phrase"]

In [251]:
# with open('data_env.json', 'w') as outfile:
#     json.dump(data, outfile)
    
with open('data_cc.json', 'w') as outfile:
    json.dump(data, outfile)

In [252]:
# x = topic_df.loc[topic_df['idea'] == 'Public Art']
# y = x.loc[x['dominant_topic'] == 1]

# len(y)

In [253]:
# for i, row in y.iterrows():
#     print(row.comment)

# Create proposal wise JSON

In [261]:
df = pd.read_json('data_cc.json')
# df = pd.read_json('data_env.json')

In [260]:
df

,comment,comment_id,discourse,discussion_id,discussion_name,dominant_topic,emotion,lemmas,related_to,topic_keywords,topic_phrase,user_id
0,A structure or monument at the Nudillo could i...,0,elaboration,1,Build a tower,1,neutral,"[incorporate, play, chime, chime, emanate, well]",Comment on the Idea,"[mobility, light, chime, village, comment, nei...",increased mobility,3
1,The built form should be lit at night.,1,humor,1,Build a tower,1,excited,"[build, form, light, night]",Comment on the Idea,"[mobility, light, chime, village, comment, nei...",increased mobility,3
2,"More than a plaza, the space can be activated ...",2,elaboration,1,Build a tower,2,excited,"[activate, landscape, place, food, truck, near...",Comment on the Idea,"[place, city, create, shade, urbanistically, e...",their impact,3
3,The structure could incorporate an observation...,3,elaboration,1,Build a tower,0,neutral,"[structure, incorporate, observation, deck, ec...",Comment on the Idea,"[structure, increase, height, transportation, ...",color realms,3
4,I think its a great idea,4,appreciation,1,Build a tower,0,happiness,"[think, great]",Comment on the Idea,"[structure, increase, height, transportation, ...",color realms,27
5,I want the color of the tower at El Nudillo to...,5,answer,1,Build a tower,4,excited,"[want, gray, art, design, bottom]",What color(s) would you want a tower at El Nud...,"[impact, area, community, good, building, agre...",your fancy lights,27
6,Any type of color realms of blue green and san...,6,answer,1,Build a tower,0,excited,"[type, color, realm, blue, sand, water, hike, ...",What color(s) would you want a tower at El Nud...,"[structure, increase, height, transportation, ...",color realms,120
7,I am not sure how it will impact safety,7,answer,1,Build a tower,2,concern,"[sure, impact, safety]",Evaluate Impact: Safety,"[place, city, create, shade, urbanistically, e...",their impact,17
8,how do you think......(building a tower) might...,8,question,1,Build a tower,4,concern,"[think, tower, impact]",Evaluate Impact: Safety,"[impact, area, community, good, building, agre...",your fancy lights,17
9,Not at all.,9,answer,1,Build a tower,0,neutral,[],Evaluate Impact: Safety,"[structure, increase, height, transportation, ...",color realms,20


In [264]:
def get_blue(total, count):
    blueSpectrum = ["#f6feff","#f7fbff","#deebf7","#c6dbef", "#9ecae1", "#6baed6", "#4292c6", "#2171b5", "#08519c", "#08306b"]
    color = round((((count / total)) * 2) * 9) if (round((((count / total)) * 2) * 9)) < 10 else 9
    return blueSpectrum[color]
        
prop_data = []
    
for i in Counter(df["discussion_name"]).keys(): 
    print(i)
    rows = df.loc[df["discussion_name"] == i]
    topic_coverage = round(len(rows) * .15)
#     topic_coverage = round(len(rows) * .10)
    prop_data_packet = {
        "discussion_id" :  rows["discussion_id"].tolist()[0],
        "discussion_name": rows["discussion_name"].tolist()[0],
        "number_of_comments": len(rows),
        "topics": [],
        "discourses": [],
        "comments" : []
    }
    
    topic_sets = []
    for j in Counter(rows["topic_phrase"]).keys():
        topic_rows = rows.loc[rows["topic_phrase"] == j]
        if(len(topic_rows) >= topic_coverage):
            topic_set_elements = {
                "topic_phrase" : j,
                "topic_keywords" : topic_rows["topic_keywords"].tolist()[0],
                "topic_sent_count" : len(topic_rows)
            }
            topic_sets.append(topic_set_elements)
    prop_data_packet["topics"] = topic_sets
    
    discourse_sets = []
    discourses = ["question", "answer", "announcement", "elaboration", "appreciation", "agreement", "disagreement", "negativereaction", "humor", "other"]
    for j in discourses:
        discourse_set_elements = {
            "discourse_name" : j,
            "count" : Counter(rows["discourse"])[j] if Counter(rows["discourse"])[j] > 0 else 0,
            "color" : get_blue(len(rows), Counter(rows["discourse"])[j])
        }
        discourse_sets.append(discourse_set_elements)
    prop_data_packet["discourses"] = discourse_sets
    
    comment_sets = []
    for idx, row in rows.iterrows():
        comment_set_element = {
            "comment_id": row["comment_id"],
            "comment": row["comment"],
            "related_to": row["related_to"],
            "discussion_id": row["discussion_id"],
            "discussion_name": row["discussion_name"],
            "user_id": row["user_id"],
            "emotion": row["emotion"],
            "discourse": row["discourse"],
            "dominant_topic": row["dominant_topic"],
            "topic_phrase": row["topic_phrase"],
            "topic_keywords": row["topic_keywords"],
            "lemmas": row["lemmas"]
        }
        comment_sets.append(comment_set_element)
    prop_data_packet["comments"] = comment_sets
    
    prop_data.append(prop_data_packet)

Build a tower
Public Art
Roundabout
Huge Fountain
Interactive Art
EL Parquecito
Pedestrian Destination
Promenade Real
Platform connected with bridges
Triangle platform with 3 pillars and flat bridge
Zipline
Transit Hub
Gateway
Connecting the past to the future
Cart before the Horse
Call the Area Something Else.
The Diamond
The Lyre
EV / Barrio Market


In [265]:
prop_df = pd.DataFrame(prop_data)

In [266]:
prop_df.to_json('mat_data_cc.json', orient = "records")

# prop_df.to_json('mat_data_env.json', orient = "records")

In [267]:
len(prop_df)

19